# HW02 - Visualizing features of a pretrained VGG

In this homework, we are going to try to visualize what neurons encode through optimization.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import models, transforms
from tqdm.notebook import tqdm
import urllib.request

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using gpu: %s " % torch.cuda.is_available())

In [ ]:
%%html

For the background of ipywidget

<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

First, load the pretrained VGG model with torchvision, and print the architecture of the model.


In [ ]:
model_vgg = models.vgg16(weights="DEFAULT", progress=True)
print(model_vgg)

# Part A - Visualizing the convolution filters

First, plot all the filters for the red channel of the first convolutional layer (there should be 64 filters in total). Can you find filters that seem to encode edges? Is this method useful for other layers?


In [ ]:
first_conv_layer = model_vgg.features[0]
red_channel_filters = first_conv_layer.weight.data[:, 0]
filters_count = len(red_channel_filters)
plot_cols_rows = int(np.ceil(np.sqrt(filters_count)))
plt.figure(figsize=(10, 10))
for i in range(filters_count):
    plt.subplot(plot_cols_rows, plot_cols_rows, i + 1)
    plt.imshow(red_channel_filters[i], cmap="gray")
    plt.title(f"Filter {i}")
plt.tight_layout()
plt.show()

<div style="border: 1px solid #0d0; padding: 10px; width: auto">

There are many filters which seem to encode edges or close to edges in the first convolutional layer `model_vgg.features[0]`:

- Vertical edges: filters 0, 6, 17, 26...
- Horizontal edges: filters 5, 9, 21...
- And even diagonal edges: filters 8, 20, 22...

Then, if we look for example at the third convolutional layer `model_vgg.features[5]`, we don't really see anymore that many filters which seem to encode edges. From a human point of view, it is hard to understand what an individual filter from this layer is optimized for.

</div>


# Part B - Visualizing channel activations through image optimization

## B.1 - First implementation

Create a module `ChannelActivation(layer, channel)` that returns the average activation (i.e. output value) of channel `channel` of layer `layer` of the VGG features.


In [ ]:
class ChannelActivation(nn.Module):
    def __init__(self, layer, channel):
        super(ChannelActivation, self).__init__()
        self.layer = layer
        self.channel = channel
        self.model_vgg_gpu = model_vgg.to(device)
        self.model_vgg_gpu.eval()

    def forward(self, x):
        output = x
        for i in range(self.layer + 1):
            output = self.model_vgg_gpu.features[i](output)
        output_channel = output[:, self.channel]
        mean_activation = torch.mean(output_channel)
        return mean_activation

Our objective is to find which patterns are recognized by a given channel. To do so, we will follow the approach of [this Distill article](https://distill.pub/2017/feature-visualization/) and find images that lead to the highest possible channel activation.

First, create a random (colored) image of size 128x128, initialized with value at random between 0.4 and 0.6 (i.e. grey + small perturbation). Then, perform 200 steps of Adam (with lr=0.01) to maximize the activation of channel 4 of layer 1. Plot the image after 0, 10, 50, 100 and 200 iterations. You should see a pink saturated image with several horizontal lines, indicating that the channel probably recognizes horizontal edges.

**NB1:** Careful, by default, optimizers minimize their objective, not maximize it!

**NB2:** The parameters given to an optimizer should be on the cpu. If you use a gpu, you thus need to keep two versions of the image: 1) a cpu version given to the optimizer, and 2) a gpu version, created at each iteration of the optimization, and used to compute the gradient.


In [ ]:
def image_activation_optimization(layer, channel, clip=False):
    input_image_cpu = (0.2 * torch.rand(1, 3, 128, 128) + 0.4).requires_grad_()
    saved_steps = [0, 10, 50, 100, 200]
    saved_images = [np.copy(input_image_cpu.detach())]

    model = ChannelActivation(layer, channel).to(device)
    optimizer = torch.optim.Adam([input_image_cpu], lr=0.01, maximize=True)

    model.eval()
    steps = 200
    for step in tqdm(range(1, steps + 1), desc="step"):
        input_image_gpu = input_image_cpu.to(device)
        optimizer.zero_grad()
        mean_activation = model(input_image_gpu).to(device)
        mean_activation.backward()
        optimizer.step()

        # Clip the value of the pixels to avoid saturation
        if clip:
            input_image_cpu.data = torch.clamp(input_image_cpu.data, 0.0, 1.0)

        # Store the new image
        if step == saved_steps[len(saved_images)]:
            saved_images.append(np.copy(input_image_cpu.detach()))

    instances = len(saved_steps)
    cols = int(np.ceil(np.sqrt(instances)))
    rows = (cols - 1) // 2 + 1
    plt.figure(figsize=(15, 15 * rows / cols))
    for i, (step, image) in enumerate(zip(saved_steps, saved_images)):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(image[0].transpose(1, 2, 0))
        plt.title(f"Step {step}")
    plt.show()

    return input_image_cpu[0].detach().numpy().transpose(1, 2, 0)


layer = 1
channel = 4
activation_image = image_activation_optimization(layer, channel)

## B.2 - Improving stability with clipping and normalization

Compute the highest and lowest values of the image. What is the issue?


In [ ]:
image_min_value = np.min(activation_image)
image_max_value = np.max(activation_image)

print(f"The lowest value of the image is {image_min_value}")
print(f"The highest value of the image is {image_max_value}")

<div style="border: 1px solid #0d0; padding: 10px; width: auto">

The issue is that the model doesn't limit itself to values between 0 and 1.

</div>


To avoid (over) saturation, clip the image pixels to $[0.2,0.8]$ after each optimization step using `input_image.data = input_image.data.clip(0.2, 0.8)`. You should now see several clear horizontal lines in a blue background.


In [ ]:
layer = 1
channel = 4
image_cpu = image_activation_optimization(layer, channel, clip=True)

image_min_value = np.min(image_cpu)
image_max_value = np.max(image_cpu)

print(f"The lowest value of the image is {image_min_value}")
print(f"The highest value of the image is {image_max_value}")

One issue with our current code, is that VGG was trained on **normalized** images, and thus is not adapted to our input image. To normalize the image, we will use **transforms**.

Create a function `create_activation_image(layer, channel, transform=None, image_size=128, show_steps=False)` that maximizes the corresponding channel activation on an image of size `image_size`, and first applies `transform` to the image before computing the gradient of the activation. The function should return the final image after 200 steps, and plot intermediate images for the steps 0,10,50,100,200 if `show_steps=True`.

Then, test your function with `transform=transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])`. Is this better? You should now see a horizontal pattern with lines.


In [ ]:
def create_activation_image(
    layer, channel, transform=None, image_size=128, show_steps=False, progress_bar=True
):
    input_image_cpu = (
        0.2 * torch.rand(1, 3, image_size, image_size) + 0.4
    ).requires_grad_()
    if show_steps:
        saved_steps = [0, 10, 50, 100, 200]
        saved_images = [np.copy(input_image_cpu.detach())]

    model = ChannelActivation(layer, channel).to(device)
    optimizer = torch.optim.Adam([input_image_cpu], lr=0.01, maximize=True)

    model.eval()
    steps = 200
    if progress_bar:
        iterator = tqdm(range(1, steps + 1), desc="step")
    else:
        iterator = range(1, steps + 1)
    for step in iterator:
        input_image_gpu = transform(input_image_cpu.to(device))
        optimizer.zero_grad()
        mean_activation = model(input_image_gpu).to(device)
        mean_activation.backward()
        optimizer.step()

        # Clip the value of the pixels to avoid saturation
        input_image_cpu.data = torch.clamp(input_image_cpu.data, 0.2, 0.8)

        # Store the new image
        if show_steps and (step == saved_steps[len(saved_images)]):
            saved_images.append(np.copy(input_image_cpu.detach()))

    # Show the image at different steps
    if show_steps:
        instances = len(saved_steps)
        cols = int(np.ceil(np.sqrt(instances)))
        rows = (cols - 1) // 2 + 1
        plt.figure(figsize=(15, 15 * rows / cols))
        for i, (step, image) in enumerate(zip(saved_steps, saved_images)):
            plt.subplot(rows, cols, i + 1)
            plt.imshow(image[0].transpose(1, 2, 0))
            plt.title(f"Step {step}")
        plt.show()

    return input_image_cpu[0].detach().numpy().transpose(1, 2, 0)


layer = 1
channel = 4
transform = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
image_size = 256
show_steps = True
activation_image = create_activation_image(
    layer, channel, transform, image_size, show_steps
)

Now test your function on channel 0 of layer 20. The pattern that appears should vagely resemble fish scales.


In [ ]:
layer = 20
channel = 0
transform = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
image_size = 256
show_steps = True
activation_image = create_activation_image(
    layer, channel, transform, image_size, show_steps
)

## B.3 - Transformation robustness

Large neural network are prone to adversarial attacks, i.e. a small well-crafted additive noise can dramatically change the output of the model, and thus lead to incorrect classification. For our purpose, this is an issue, as the optimization algorithm may find such very specific noise instead of more valuable visual patterns.

To avoid this issue and further improve our images, we are thus going to apply small random perturbations to the image before computing the gradient. This will prevent the optimizer from optimizing the noise, and overall increase the stability of our process.

To do so, add a composition of several transforms (before the normalization):

1.  A small pixel noise with `transforms.Lambda(lambda x: x + 0.001 * (2 * torch.rand_like(x) - 1))`
2.  A random affine transform with `transforms.RandomAffine(degrees=5, translate=(0.1,0.1), scale=(0.9,1.1))`
3.  A random crop of size 96 (to reduce the size of the image)
4.  Random local fluctations with `transforms.ElasticTransform(alpha=50.)`

Compare the activation images with and without these random transformations. Is the pattern more visible?


In [ ]:
layer = 1
channel = 4
transform = transforms.Compose(
    [
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.Lambda(lambda x: x + 0.001 * (2 * torch.rand_like(x) - 1)),
        transforms.RandomAffine(degrees=5, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        transforms.RandomCrop(192),
        transforms.ElasticTransform(alpha=50.0),
    ]
)
image_size = 256
show_steps = True
activation_image = create_activation_image(
    layer, channel, transform, image_size, show_steps
)

In [ ]:
layer = 20
channel = 0
transform = transforms.Compose(
    [
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.Lambda(lambda x: x + 0.001 * (2 * torch.rand_like(x) - 1)),
        transforms.RandomAffine(degrees=5, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        transforms.RandomCrop(192),
        transforms.ElasticTransform(alpha=50.0),
    ]
)
image_size = 256
show_steps = True

activation_image = create_activation_image(
    layer, channel, transform, image_size, show_steps
)

To see what the transformation is doing to an image, apply the random transformations (without normalization) to the following simple image, and show 5 randomly transformed images.


In [ ]:
sample_image = 0.3 * torch.ones(3, 256, 256)
sample_image[0, :, 40:80] += 0.7
sample_image[1, 10:20, :] += 0.5
sample_image[2, 150:, :] += 0.5
plt.imshow(transforms.functional.to_pil_image(sample_image))
plt.show()

transform = transforms.Compose(
    [
        transforms.Lambda(lambda x: x + 0.001 * (2 * torch.rand_like(x) - 1)),
        transforms.RandomAffine(degrees=5, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        transforms.RandomCrop(192),
        transforms.ElasticTransform(alpha=50.0),
    ]
)

instances = 6
cols = int(np.ceil(np.sqrt(instances)))
rows = (cols - 1) // 2 + 1
plt.figure(figsize=(15, 15 * rows / cols))
for i in range(instances):
    transformed_sample_image = transforms.functional.to_pil_image(
        transform(sample_image)
    )
    plt.subplot(rows, cols, i + 1)
    plt.imshow(transformed_sample_image)
plt.show()

## B.4 - Final visualization

Finally, show the activation images for the first 5 channels of layers [1, 10, 20, 30]. You should be able to see a gradual complexification of the patterns.

**PS1:** Our method seems unable to find meaningful patterns for the last layer. One issue is probably that the random crop imposes that all regions on the image look similar (as they all should have a high channel activation), thus preventing larger and more complex patterns to emerge from the optimization.

**PS2:** You can also try other layers and channels to find interesting patterns!


In [ ]:
layers = [1, 10, 20, 30]
channels = [0, 1, 2, 3, 4, 5]
transform = transforms.Compose(
    [
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.Lambda(lambda x: x + 0.001 * (2 * torch.rand_like(x) - 1)),
        transforms.RandomAffine(degrees=5, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        transforms.RandomCrop(96),
        transforms.ElasticTransform(alpha=50.0),
    ]
)
image_size = 256
show_steps = False

# Compute the activation images
activation_images = np.zeros((len(layers), len(channels), image_size, image_size, 3))
for i, layer in enumerate(tqdm(layers, position=0, desc="layer", colour="green")):
    for j, channel in enumerate(
        tqdm(channels, position=1, desc="channel", leave="False", colour="red")
    ):
        activation_images[i, j] = create_activation_image(
            layer, channel, transform, image_size, show_steps, progress_bar=False
        )

In [ ]:
# Display the images
for i, layer in enumerate(layers):
    instances = len(channels)
    cols = int(np.ceil(np.sqrt(instances)))
    rows = (cols - 1) // 2 + 1
    plt.figure(figsize=(15, 15 * rows / cols))
    plt.suptitle(f"Layer {layer}")
    for j, channel in enumerate(channels):
        plt.subplot(rows, cols, j + 1)
        plt.imshow(activation_images[i, j])
        plt.title(f"Channel {channel}")
    plt.tight_layout()
    plt.show()

In [ ]:
# model = models.vgg16(weights="DEFAULT", progress=True)
model = models.convnext_tiny(weights="DEFAULT", progress=True)
model.layers = model.features
print(model.layers)

In [ ]:
# URL to the ImageNet class labels JSON file
url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"

# Download the file and load the class labels
class_labels = []
with urllib.request.urlopen(url) as f:
    class_labels = [line.decode("utf-8").strip() for line in f.readlines()]

# Print the first 10 class labels
print("First 10 class labels:", class_labels[:10])

In [ ]:
from utils import create_activation_image

layers = [len(model.layers) - 1]
channels = list(range(4))
channels_names = [class_labels[i] for i in channels]
steps = 400
lr = 0.04
show_steps = True
input_shape = (3, 236, 236)

# activation_images_transform = None
activation_images_transform = transforms.Compose(
    [
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.Lambda(lambda x: x + 0.001 * (2 * torch.rand_like(x) - 1)),
        transforms.RandomAffine(degrees=5, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        transforms.RandomCrop(224),
        transforms.ElasticTransform(alpha=50.0),
    ]
)


# Compute the activation images
activation_images = np.zeros((len(layers), len(channels), *input_shape))
for i, layer in enumerate(tqdm(layers, position=0, desc="layer", colour="green")):
    for j, channel in enumerate(
        tqdm(channels, position=1, desc="channel", colour="red")
    ):
        activation_images[i, j] = create_activation_image(
            model=model,
            layer=layer,
            channel=channel,
            input_mean=0,
            input_std=1,
            steps=steps,
            lr=lr,
            show_steps=show_steps,
            transform=activation_images_transform,
            input_shape=input_shape,
            progress_bar=True,
        )

In [ ]:
# Display the images
for i, layer in enumerate(layers):
    instances = len(channels)
    cols = int(np.ceil(np.sqrt(instances)))
    rows = int(np.ceil(instances / cols))
    plt.figure(figsize=(15, 15 * rows / cols))
    plt.suptitle(f"Layer {layer}")
    for j, (channel, channel_name) in enumerate(zip(channels, channels_names)):
        plt.subplot(rows, cols, j + 1)
        plt.imshow(activation_images[i, j].transpose(1, 2, 0).squeeze(), cmap="gray")
        plt.title(channel_name)
        plt.axis("off")
    plt.tight_layout()
    plt.show()

In [ ]:
import torch
import torchvision.models as models
import json
import urllib.request

# Load the pre-trained VGG16 model
vgg16 = models.vgg16(pretrained=True)

# URL to the ImageNet class labels JSON file
url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"

# Download the file and load the class labels
class_labels = []
with urllib.request.urlopen(url) as f:
    class_labels = [line.strip() for line in f.readlines()]

# Print the first 10 class labels
print("First 10 class labels:", class_labels[:10])

# Example: Predict the class of a random input tensor
input_tensor = torch.randn(1, 3, 224, 224)  # Random input tensor
output = vgg16(input_tensor)
_, predicted_idx = torch.max(output, 1)
predicted_class = class_labels[predicted_idx.item()]

print("Predicted class:", predicted_class)